In [ ]:
import pandas as pd
import numpy as np

print('Версия pandas:', pd.__version__)

Версия pandas: 2.1.4


# Пропущенные (отсутствующие) значения. Часть 2
# Missing values - NaN, pd.NA, None

## Обнаружение, удаление и замена отсутствующих значений

### План урока
    
#### 1. Обнаружение и подсчет отсутствующих значений (NaN)
    
    1.1 Проверка, являются ли ВСЕ элементы пропусками (NaN)
    1.2 Проверка, является ли ХОТЯ БЫ один элемент пропуском (NaN)
    1.3 Подсчет количества значений NaN (пропусков) в каждой строке и каждом столбце
    1.4 Подсчет количества непропущенных (существующих) значений в каждой строке и каждом столбце
    1.5 Подсчет ОБЩЕГО количества значений NaN (пропусков)
    1.6 Подсчет ОБЩЕГО количества непропущенных (существующих) значений
    1.7 Проверка, содержит ли структура DataFrame хотя бы один пропуск значения (NaN)
    
#### 2. Удаление пропущенных значений с помощью метода dropna()
    
    2.1 Удаление строк и столбцов, в которых ВСЕ элементы - NaN
    2.2 Удаление строк и столбцов, в которых ХОТЯ БЫ ОДИН элемент - NaN
    2.3 Удаление строк и столбцов, в соответствии с пороговым количеством существующих значений - параметр thresh
    2.4 Удаление NaN с учетом определенных строк/столбцов: подмножество (subset)
    2.5 Сохранение изменений в исходном объекте с помощью inplace (метод dropna)
    2.6 Метод dropna для pd.Series
    
#### 3. Замена пропущенных значений NaN с помощью метода fillna() (+ ffill(), bfill() и combine_first())
    
    3.1 Замена пропуска NaN общим значением
    3.2 Замена пропусков NaN разными значениями для каждого столбца
        3.2.1 Используем словарь dict для заполнения пропусков
        3.2.2 Используем структуру Series для заполнения пропусков
        3.2.3 Используем структуру DataFrame для заполнения пропусков
        3.2.4 Метод combine_first (для объектов Series и DataFrame)
    3.3 Замена пропусков NaN средним, медианой или модой для каждого столбца
    3.4 Замена NaN соседними значениями: методы ffill() и bfill()
    3.5 Параметр method в fillna() и устаревшие методы pad() и backfill()
    3.6 Сохранение изменений в исходном объекте с помощью inplace (метод fillna)
    3.7 Методы fillna(), ffill() и bfill() объекта Series
    

In [ ]:
df = pd.read_csv('files/sample_1_nan.csv')
df

# 1. Обнаружение и подсчет отсутствующих значений (NaN)

In [ ]:
# проверяем пропуски с помощью метода .isnull()
# использовав df.isna(), мы получим аналогичный результат
# использовав df.notnull() или df.notna(), мы получим обратную булеву маску, где на позициях пропусков будет False
df.isnull()

In [ ]:
# получаем сводную информацию о структуре
df.info()

## 1.1 Проверка, являются ли ВСЕ элементы пропусками (NaN)
## Используем isnull+all для столбцов и isnull+all(axis=1) для строк

_Вызвав метод all() для результата isnull(), вы можете проверить, все ли элементы в каждой строке и столбце являются NaN. Метод all() возвращает True, если все элементы в каждой строке и столбце равны True.По умолчанию эта функция применяется к столбцам. Если axis=1, то применяется к строкам._

In [ ]:
# Вызвав all() для результата isnull(), проверяем, все ли элементы в каждом столбце являются NaN.
# в последнем столбце other все значения нулевые (пропущены)
# указываем all(axis=0, по умолчанию)

display(df.isnull(),
        df.isnull().all())

In [ ]:
# Вызвав all() для результата isnull(), проверяем, все ли элементы в каждой строке являются NaN.
# во второй строке (с индексом 1) все значения нулевые (пропущены)
# указываем all(axis=1)
df.isnull().all(axis=1)

## 1.2 Проверка, является ли ХОТЯ БЫ один элемент пропуском (NaN)
## Используем isnull+any для столбцов и isnull+any(axis=1) для строк

In [ ]:
# Вызвав any() для результата isnull(), проверяем, есть ли хотя бы одно значение TRUE (пропуск NaN) в столбце булева датафрейма
# указываем any(axis=0, по умолчанию)
display(df.isnull(),
        df.isnull().any())

In [ ]:
# указываем any(axis=1), чтобы сделать проверку по каждой строке булева датафрейма
df.isnull().any(axis=1)

## 1.3 Подсчет количества NaN (пропусков) в каждой строке и каждом столбце

## Используем isnull + sum для столбцов и isnull + sum(axis=1) для строк

In [ ]:
display(df.isnull(),
        # количество пропусков по каждому столбцу
        df.isnull().sum(),
        # количество пропусков по каждой строке
        df.isnull().sum(axis=1))

## 1.4 Подсчет количества не-NaN (существующих) значений в каждой строке и каждом столбце

#### Способ 1 (для столбцов) - метод .info()

In [ ]:
df.info()

#### Способ 2 (для столбцов и строк). Используем .notnull() + .sum()
    в методе .sum() axis=0 - для столбцов (по умолчанию) или axis=1 - для строк

In [ ]:
display(df.notnull(),
        # количество пропусков по каждому столбцу
        df.notnull().sum(),
        # количество пропусков по каждой строке
        df.notnull().sum(axis=1))

#### Способ 3 (для столбцов и строк). Используем .count()
    в методе .count() axis=0 - для столбцов (по умолчанию) или axis=1 - для строк

In [ ]:
display(df,
        # количество пропусков по каждому столбцу
        df.count(),
        # количество пропусков по каждой строке
        df.count(axis=1))

## 1.5 Подсчет ОБЩЕГО количества значений NaN (пропусков)

In [ ]:
# получаем все данные булева датафрейма в виде массива NumPy
display(df.isnull(),
        df.isnull().values,
        type(df.isnull().values))

In [ ]:
df.isnull().values.sum()

## 1.6 Подсчет ОБЩЕГО количества непропущенных (существующих) значений

In [ ]:
df.count().sum()

In [ ]:
# тот же результат при использовании df.notna().values.sum()
df.notnull().values.sum()

## 1.7 Проверка, содержит ли структура DataFrame хотя бы один пропуск значения (NaN)

In [ ]:
# если общее количество NaN не равно нулю, значит, DataFrame содержит хотя бы один NaN.
df.isnull().values.sum() != 0

In [ ]:
# атрибут size показывает количество всех элементов структуры df
# Если общее количество NaN равно атрибуту size, то все элементы структуры являются NaN.
print(df.size)
print(df.isnull().values.sum())
print(df.isnull().values.sum() == df.size)

# 2. Удаление пропущенных значений с помощью метода dropna()

## 2.1 Удаление строк и столбцов, в которых ВСЕ элементы - NaN

#### Для удаления СТРОК, полностью состоящих из NaN, используем параметр how='all'

In [ ]:
df.dropna(how='all')

#### Для удаления СТОЛБЦОВ, полностью состоящих из NaN, используем параметр how='all' вместе с параметром axis=1

In [ ]:
df.dropna(how='all', axis=1)

In [ ]:
# print(df.dropna(how='all', axis=[0, 1]))
# TypeError: supplying multiple axes to axis is no longer supported.

df.dropna(how='all').dropna(how='all', axis=1)

## 2.2 Удаление строк и столбцов, в которых ХОТЯ БЫ ОДИН элемент - NaN
    (вариант по умолчанию - how='any' (default))

In [ ]:
df2 = df.dropna(how='all').dropna(how='all', axis=1)

In [ ]:
df2.dropna(how='any')

In [ ]:
df2.dropna()

In [ ]:
df2.dropna(axis=1)

## 2.3 Удаление строк и столбцов, в соответствии с пороговым количеством существующих значений (не-NaN элементов)
    используем параметр thresh

In [ ]:
# строки, содержащие более трех непропущенных значений, остаются, а остальные строки удаляются.
df.dropna(thresh=3)

In [ ]:
# остаются только столбцы, содержащие более трех непустых значений
df.dropna(thresh=3, axis=1)

## 2.4 Удаление NaN с учетом определенных строк/столбцов: подмножество (subset)

In [ ]:
# будут удалены строки, содержащие NaN в столбцах, указанных в аргументе subset
df.dropna(subset=['age'])

In [ ]:
# будут удалены строки, содержащие NaN в столбцах, указанных в аргументе subset
df.dropna(subset=['age', 'state'])

In [ ]:
# Если параметр how имеет значение 'all', будут удалены строки с содержанием NaN во ВСЕХ указанных в subset столбцах.
df.dropna(subset=['age', 'state'], how='all')

In [ ]:
# если axis=1 (или axis='columns')
# удаляются СТОЛБЦЫ с NaN во всех указанных в subset СТРОКАХ

display(df,
        df.dropna(subset=[0, 4], axis=1))

In [ ]:
# колонка, чтобы быть удаленной, должна содержать NaN как в строке с индексом 0, так и в строке с индексом 4
df.dropna(subset=[0, 4], axis=1, how='all')

_При использовании метода .dropna() вместе с параметром subset может возникнуть ОШИБКА, если, например, указано несуществующее имя строки или столбца, или, если задать axis=1, но указать имена столбцов, а не строк, или оставить axis по умолчанию, но указать имена строк, а не столбцов._

In [ ]:
# print(df.dropna(subset=['age', 'state', 'xxx']))
# KeyError: ['xxx']

# print(df.dropna(subset=['age', 'state'], axis=1))
# KeyError: ['age', 'state']

## 2.5 Сохранение изменений в исходном объекте с помощью inplace (метод dropna)

In [ ]:
df.dropna(subset=['age'], inplace=True)
df

## 2.6 Метод dropna для объекта Series

In [ ]:
s = pd.read_csv('files/sample_1_nan.csv')['age']
print(s)
print()
s.dropna()

In [ ]:
# единственным допустимым аргументом для метода dropna() объекта Series является inplace
s.dropna(inplace=True)
s

# 3. Замена пропущенных значений NaN с помощью метода fillna()

    Метод .fillna()
    Первый параметр value - значение, на которое мы заменяем отсутствующие значения NaN
    Может принимать scalar, dict, Series, or DataFrame

In [ ]:
df = pd.read_csv('files/sample_1_nan.csv')
df

,name,age,state,point,other
0,Alice,24.0,NY,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,Charlie,NaN,CA,NaN,NaN
3,Dave,68.0,TX,70.0,NaN
4,Ellen,NaN,CA,88.0,NaN
5,Frank,30.0,NaN,NaN,NaN


## 3.1 Замена пропуска NaN общим значением

In [ ]:
# Если указать скалярное значение в качестве первого аргумента (value) в fillna(),
# все значения NaN будут заменены на это значение
df.fillna(0)

,name,age,state,point,other
0,Alice,24.0,NY,0.0,0.0
1,0,0.0,0,0.0,0.0
2,Charlie,0.0,CA,0.0,0.0
3,Dave,68.0,TX,70.0,0.0
4,Ellen,0.0,CA,88.0,0.0
5,Frank,30.0,0,0.0,0.0


In [ ]:
# изменяем тип данных
df_int_age = df.fillna(0)
print(df_int_age['age'].astype('int64'))

df_int_age['age']=df_int_age['age'].astype('int64')
df_int_age

0    24
1     0
2     0
3    68
4     0
5    30
Name: age, dtype: int64


,name,age,state,point,other
0,Alice,24,NY,0.0,0.0
1,0,0,0,0.0,0.0
2,Charlie,0,CA,0.0,0.0
3,Dave,68,TX,70.0,0.0
4,Ellen,0,CA,88.0,0.0
5,Frank,30,0,0.0,0.0


## 3.2 Замена пропусков NaN разными значениями для каждого столбца

## 3.2.1 Используем словарь dict для заполнения пропусков

    Передать первым аргументом в метод .fillna() словарь вида...

    {имя_столбца_1: значение для замены NaN в столбце_1, имя_столбца_2: значение для замены NaN в столбце_2}

In [ ]:
df.fillna({'name': 'XXX', 'age': 20, 'ZZZ': 100})

,name,age,state,point,other
0,Alice,24.0,NY,NaN,NaN
1,XXX,20.0,NaN,NaN,NaN
2,Charlie,20.0,CA,NaN,NaN
3,Dave,68.0,TX,70.0,NaN
4,Ellen,20.0,CA,88.0,NaN
5,Frank,30.0,NaN,NaN,NaN


### Вы также можете указать вместо словаря объект Series. Метки Series соответствуют ключам словаря.

## 3.2.2 Используем структуру Series для заполнения пропусков

In [ ]:
# можем указать вместо словаря объект Series. Метки Series соответствуют ключам словаря

s_for_fill = pd.Series(['XXX', 0, 100], index=['name', 'age', 'ZZZ'])
print(s_for_fill)
df.fillna(s_for_fill)

name    XXX
age       0
ZZZ     100
dtype: object


,name,age,state,point,other
0,Alice,24.0,NY,NaN,NaN
1,XXX,0.0,NaN,NaN,NaN
2,Charlie,0.0,CA,NaN,NaN
3,Dave,68.0,TX,70.0,NaN
4,Ellen,0.0,CA,88.0,NaN
5,Frank,30.0,NaN,NaN,NaN


## 3.2.3 Используем структуру DataFrame для заполнения пропусков

In [ ]:
df

,name,age,state,point,other
0,Alice,24.0,NY,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,Charlie,NaN,CA,NaN,NaN
3,Dave,68.0,TX,70.0,NaN
4,Ellen,NaN,CA,88.0,NaN
5,Frank,30.0,NaN,NaN,NaN


In [ ]:
df_fill= pd.read_csv('files/filling_df')
df_fill

,name,age,point,sex
0,Alice,24,78,F
1,John,23,86,M
2,Charlie,45,67,M
3,Dave,68,70,M
4,Ellen,36,88,F


In [ ]:
df.fillna(df_fill)

,name,age,state,point,other
0,Alice,24.0,NY,78.0,NaN
1,John,23.0,NaN,86.0,NaN
2,Charlie,45.0,CA,67.0,NaN
3,Dave,68.0,TX,70.0,NaN
4,Ellen,36.0,CA,88.0,NaN
5,Frank,30.0,NaN,NaN,NaN


    Необходимо быть внимательными при этом способе заполнения пропусков.

In [ ]:
# создаем новый объект DataFrame в переменной new_df_fill, добавив к df_fill еще одну строку с индексом 5
new_df_fill=pd.concat([df_fill,
                       pd.DataFrame([['Michael', 35, 89, 'M']], index=[5], columns=['name','age','point','sex'])])
new_df_fill

,name,age,point,sex
0,Alice,24,78,F
1,John,23,86,M
2,Charlie,45,67,M
3,Dave,68,70,M
4,Ellen,36,88,F
5,Michael,35,89,M


In [ ]:
# обращаем внимание на соответствие индексов строк и столбцов при заполнении одного датафрейма другим
# в данном примере мы получили неочевидную ошибку
# в строке с индексом 5 Frank получил значение point (89), относящееся к Michael в датафрейме new_df_fill
df.fillna(new_df_fill)

,name,age,state,point,other
0,Alice,24.0,NY,78.0,NaN
1,John,23.0,NaN,86.0,NaN
2,Charlie,45.0,CA,67.0,NaN
3,Dave,68.0,TX,70.0,NaN
4,Ellen,36.0,CA,88.0,NaN
5,Frank,30.0,NaN,89.0,NaN


In [ ]:
# во избежание ошибок используйте уникальные данные (например, id)
df['id']=['P001', np.nan, 'P003', 'P004', 'P005', 'P006']
new_df_fill['id']=['P001', 'P002', 'P003', 'P004', 'P005', 'P007']
# устанавливаем колонки с id в качестве индексов строк
# используем метод set_index('id')
df.set_index('id', inplace=True)
new_df_fill.set_index('id', inplace=True)
display(df, new_df_fill)

,name,age,state,point,other
id,,,,,
P001,Alice,24.0,NY,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN
P003,Charlie,NaN,CA,NaN,NaN
P004,Dave,68.0,TX,70.0,NaN
P005,Ellen,NaN,CA,88.0,NaN
P006,Frank,30.0,NaN,NaN,NaN


,name,age,point,sex
id,,,,
P001,Alice,24,78,F
P002,John,23,86,M
P003,Charlie,45,67,M
P004,Dave,68,70,M
P005,Ellen,36,88,F
P007,Michael,35,89,M


In [ ]:
# производим заполнение пропусков без ошибок
df.fillna(new_df_fill)

,name,age,state,point,other
id,,,,,
P001,Alice,24.0,NY,78.0,NaN
NaN,NaN,NaN,NaN,NaN,NaN
P003,Charlie,45.0,CA,67.0,NaN
P004,Dave,68.0,TX,70.0,NaN
P005,Ellen,36.0,CA,88.0,NaN
P006,Frank,30.0,NaN,NaN,NaN


## 3.2.4 Метод combine_first (для объектов Series и DataFrame)

_С помощью метода combine_first мы объединяем два объекта DataFrame (или Series), заполняя пропуски в первом объекте DataFrame (Series) ненулевыми значениями из второго объекта DataFrame._

In [ ]:
# автоматическая сортировка меток строк и столбцов
# в результат включаются столбцы и строки из второй структуры
res=df.combine_first(new_df_fill)
res

,age,name,other,point,sex,state
id,,,,,,
P001,24.0,Alice,NaN,78.0,F,NY
P002,23.0,John,NaN,86.0,M,NaN
P003,45.0,Charlie,NaN,67.0,M,CA
P004,68.0,Dave,NaN,70.0,M,TX
P005,36.0,Ellen,NaN,88.0,F,CA
P006,30.0,Frank,NaN,NaN,NaN,NaN
P007,35.0,Michael,NaN,89.0,M,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_add=pd.DataFrame([['Johnny', 23, 86, 'M', 'CA'], ['Helen', 36, 88, 'F','CA']],
                    index=['P002', 'P005'],
                    columns=['name','age','point','sex','state'])
df_add

,name,age,point,sex,state
P002,Johnny,23,86,M,CA
P005,Helen,36,88,F,CA


In [ ]:
# метод не перезаписывает данные, а именно заполняет пропуски
# в примере имя 'John' осталось без изменения,
# но пропуск значения в колонке state заполнился значением 'CA' из датафрейма df_add
res.combine_first(df_add)

,age,name,other,point,sex,state
P001,24.0,Alice,NaN,78.0,F,NY
P002,23.0,John,NaN,86.0,M,CA
P003,45.0,Charlie,NaN,67.0,M,CA
P004,68.0,Dave,NaN,70.0,M,TX
P005,36.0,Ellen,NaN,88.0,F,CA
P006,30.0,Frank,NaN,NaN,NaN,NaN
P007,35.0,Michael,NaN,89.0,M,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# так же работает для Series
df['age'].combine_first(new_df_fill['age'])

id
P001    24.0
P002    23.0
P003    45.0
P004    68.0
P005    36.0
P006    30.0
P007    35.0
NaN      NaN
Name: age, dtype: float64

## 3.3 Замена пропусков NaN средним, медианой или модой для каждого столбца

    Метод .mean()
    среднее значение - это среднее арифметическое
_среднее арифметическое легко посчитать, у него есть серьёзный недостаток: если один показатель сильно отличается от остальных, то он серьёзно искажает итоговый результат._
   
    Метод .median()
    медианное значение - это серединное значение, число, которое находится в середине ряда,
    то есть половина чисел имеют значения большие, чем медиана, а половина чисел имеют значения меньшие, чем медиана.
_если набор данных состоит из четного количества значений, то медиана — это среднее двух серединных значений_

    Метод .mode()
    Мода - это значение, наиболее часто встречающееся в наборе. В одном наборе может быть одна или несколько мод.
    Мода может быть как числовым, так и строковым значением.
    
####  Если структура DataFrame содержит НЕ только числовые столбцы, но и строки, то используя методы .mean() и .median(), обязательно указываем аргумент numeric_only=True, иначе получим TypeError
#### Поскольку метод .mode() возвращает объект DataFrame, то, чтобы получить серию с модальными значениями для последующего использования в fillna, получаем серию через iloc - df.mode().iloc[0]

In [ ]:
df = pd.read_csv('files/sample_1_nan.csv')
df

,name,age,state,point,other
0,Alice,24.0,NY,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,Charlie,NaN,CA,NaN,NaN
3,Dave,68.0,TX,70.0,NaN
4,Ellen,NaN,CA,88.0,NaN
5,Frank,30.0,NaN,NaN,NaN


#### Метод .mean()

In [ ]:
# получаем серию средних значений для каждого числового столбца
# NaN исключается из расчета, но столбцы, в которых все элементы равны NaN, остаются NaN.
# аргумент numeric_only должен быть установлен в True, чтобы включить только числовые столбцы и не получить TypeError
df.mean(numeric_only=True)

age      40.666667
point    79.000000
other          NaN
dtype: float64

In [ ]:
# заменяем пропуски средним значением по каждому числовому столбцу
# передаем полученную серию средних значений первым аргументом
df.fillna(df.mean(numeric_only=True))

,name,age,state,point,other
0,Alice,24.000000,NY,79.0,NaN
1,NaN,40.666667,NaN,79.0,NaN
2,Charlie,40.666667,CA,79.0,NaN
3,Dave,68.000000,TX,70.0,NaN
4,Ellen,40.666667,CA,88.0,NaN
5,Frank,30.000000,NaN,79.0,NaN


#### Метод .median()

In [ ]:
# получаем серию медианных значений для каждого числового столбца
df.median(numeric_only=True)

age      30.0
point    79.0
other     NaN
dtype: float64

In [ ]:
# заменяем пропуски медианными значениями
df.fillna(df.median(numeric_only=True))

,name,age,state,point,other
0,Alice,24.0,NY,79.0,NaN
1,NaN,30.0,NaN,79.0,NaN
2,Charlie,30.0,CA,79.0,NaN
3,Dave,68.0,TX,70.0,NaN
4,Ellen,30.0,CA,88.0,NaN
5,Frank,30.0,NaN,79.0,NaN


#### Метод .mode()

In [ ]:
# mode() возвращает DataFrame с одним или несколькими модальными значениями для каждого столбца
# первая строка получается как Series с помощью iloc[0].
# используем эту серию для заполнения пропусков модальным значением
# mode() также может работать со строками
display(df.mode(),
        df.mode().iloc[0])

,name,age,state,point,other
0,Alice,24.0,CA,70.0,NaN
1,Charlie,30.0,NaN,88.0,NaN
2,Dave,68.0,NaN,NaN,NaN
3,Ellen,NaN,NaN,NaN,NaN
4,Frank,NaN,NaN,NaN,NaN


name     Alice
age       24.0
state       CA
point     70.0
other      NaN
Name: 0, dtype: object

In [ ]:
# заменяем пропуски модой
df.fillna(df.mode().iloc[0])

,name,age,state,point,other
0,Alice,24.0,NY,70.0,NaN
1,Alice,24.0,CA,70.0,NaN
2,Charlie,24.0,CA,70.0,NaN
3,Dave,68.0,TX,70.0,NaN
4,Ellen,24.0,CA,88.0,NaN
5,Frank,30.0,CA,70.0,NaN


## 3.4 Замена NaN соседними значениями: методы ffill() и bfill()

In [ ]:
# ffill() заменяет NaN ПРЕДЫДУЩИМ допустимым значением
df.ffill()

,name,age,state,point,other
0,Alice,24.0,NY,NaN,NaN
1,Alice,24.0,NY,NaN,NaN
2,Charlie,24.0,CA,NaN,NaN
3,Dave,68.0,TX,70.0,NaN
4,Ellen,68.0,CA,88.0,NaN
5,Frank,30.0,CA,88.0,NaN


In [ ]:
# bfill() заменяет NaN СЛЕДУЮЩИМ допустимым значением
df.bfill()

,name,age,state,point,other
0,Alice,24.0,NY,70.0,NaN
1,Charlie,68.0,CA,70.0,NaN
2,Charlie,68.0,CA,70.0,NaN
3,Dave,68.0,TX,70.0,NaN
4,Ellen,30.0,CA,88.0,NaN
5,Frank,30.0,NaN,NaN,NaN


In [ ]:
# По умолчанию заменяются все последовательные значения NaN
# Аргумент limit определяет, сколько последовательных замен разрешено

df.ffill(limit=1)

,name,age,state,point,other
0,Alice,24.0,NY,NaN,NaN
1,Alice,24.0,NY,NaN,NaN
2,Charlie,NaN,CA,NaN,NaN
3,Dave,68.0,TX,70.0,NaN
4,Ellen,68.0,CA,88.0,NaN
5,Frank,30.0,CA,88.0,NaN


In [ ]:
df.bfill(limit=1)

,name,age,state,point,other
0,Alice,24.0,NY,NaN,NaN
1,Charlie,NaN,CA,NaN,NaN
2,Charlie,68.0,CA,70.0,NaN
3,Dave,68.0,TX,70.0,NaN
4,Ellen,30.0,CA,88.0,NaN
5,Frank,30.0,NaN,NaN,NaN


In [ ]:
# Установка аргумента axis в 1 или 'columns' заменяет NaN на левое или правое значение
# ffill() использует левое значение, а bfill() - правое

df.ffill(axis=1)

,name,age,state,point,other
0,Alice,24.0,NY,NY,NY
1,NaN,NaN,NaN,NaN,NaN
2,Charlie,Charlie,CA,CA,CA
3,Dave,68.0,TX,70.0,70.0
4,Ellen,Ellen,CA,88.0,88.0
5,Frank,30.0,30.0,30.0,30.0


In [ ]:
df.bfill(axis=1)

,name,age,state,point,other
0,Alice,24.0,NY,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,Charlie,CA,CA,NaN,NaN
3,Dave,68.0,TX,70.0,NaN
4,Ellen,CA,CA,88.0,NaN
5,Frank,30.0,NaN,NaN,NaN


    Ранее для этих же целей использовались методы pad() и backfill(), но сейчас они уже являются устаревшими

## 3.5 Параметр method в fillna()

In [ ]:
# Установка аргумента method в 'ffill' или 'pad' воспроизводит функциональность ffill(),
# а 'bfill' или 'backfill' дает тот же результат, что и bfill().
df.fillna(method='ffill', limit=1)
df.fillna(method='pad', limit=1)
df.fillna(method='bfill', limit=1)
df.fillna(method='backfill', limit=1)

C:\Users\iaasm\AppData\Local\Temp\ipykernel_16060\3209503515.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', limit=1)
C:\Users\iaasm\AppData\Local\Temp\ipykernel_16060\3209503515.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='pad', limit=1)
C:\Users\iaasm\AppData\Local\Temp\ipykernel_16060\3209503515.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', limit=1)
C:\Users\iaasm\AppData\Local\Temp\ipykernel_16060\3209503515.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='backfill', limit=1)


,name,age,state,point,other
0,Alice,24.0,NY,NaN,NaN
1,Charlie,NaN,CA,NaN,NaN
2,Charlie,68.0,CA,70.0,NaN
3,Dave,68.0,TX,70.0,NaN
4,Ellen,30.0,CA,88.0,NaN
5,Frank,30.0,NaN,NaN,NaN


## 3.6 Сохранение изменений в исходном объекте с помощью inplace (метод fillna)

In [ ]:
# установка аргумента inplace в True изменяет исходный объект
df.fillna(0, inplace=True)

In [ ]:
df

,name,age,state,point,other
0,Alice,24.0,NY,0.0,0.0
1,0,0.0,0,0.0,0.0
2,Charlie,0.0,CA,0.0,0.0
3,Dave,68.0,TX,70.0,0.0
4,Ellen,0.0,CA,88.0,0.0
5,Frank,30.0,0,0.0,0.0


## 3.7 Методы fillna(), ffill() и bfill() объекта Series

In [ ]:
s_1 = pd.read_csv('files/sample_1_nan.csv')['age']
s_1

0    24.0
1     NaN
2     NaN
3    68.0
4     NaN
5    30.0
Name: age, dtype: float64

In [ ]:
print(s_1.fillna(100),
      s_1.fillna({1: 100, 4: -100}),
      s_1.ffill(limit=1),
      s_1.bfill(limit=1),
      sep='\n\n')

0     24.0
1    100.0
2    100.0
3     68.0
4    100.0
5     30.0
Name: age, dtype: float64

0     24.0
1    100.0
2      NaN
3     68.0
4   -100.0
5     30.0
Name: age, dtype: float64

0    24.0
1    24.0
2     NaN
3    68.0
4    68.0
5    30.0
Name: age, dtype: float64

0    24.0
1     NaN
2    68.0
3    68.0
4    30.0
5    30.0
Name: age, dtype: float64
